<a href="https://colab.research.google.com/github/daawonn/dl_project/blob/master/5_%ED%85%90%EC%84%9C%ED%94%8C%EB%A1%9C%EC%9A%B0%EB%A5%BC%EC%9D%B4%EC%9A%A9%ED%95%9CCNN%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 연구목표

- MNIST 손글씨 이미지 데이터를 사용
- 손글씨 이미지를 분류 0 ~ 9
- 알고리즘
  - 딥러닝의 CNN을 이용하여서 손글씨를 예측하는 모델을 구축
  - 예측 모델, 정확도를 평가, 관찰
- 딥러닝 엔진 중 텐서플로우를 이용하여 구현한다


# 2. 데이터 확보/수집

In [1]:
import tensorflow as tf
%tensorflow_version 1.x

In [2]:
mnist = tf.keras.datasets.mnist.load_data(path='mnist.npz')

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
type(mnist), len(mnist[0][0]), mnist[0][1], len(mnist[1][0]), mnist[1][1]
# 훈련용 데이터 : mnist[0][0]
# 훈련용 레이블 : mnist[0][1]
# 테스트용 데이터 : mnist[1][0]
# 테스트용 레이블 : mnist[1][1]
# 레이블 데이터는 벡터화처리가 되지 않았고 분류형으로 0~9까지 배치되어있다

(tuple,
 60000,
 array([5, 0, 4, ..., 5, 6, 8], dtype=uint8),
 10000,
 array([7, 2, 1, ..., 4, 5, 6], dtype=uint8))

In [4]:
# 레이블이 이미 벡터화된 채로 제공하는 tf의 데이터셋을 활용
# 단 1.x에서만 사용가능
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./data/mnist/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [5]:
type(mnist)

tensorflow.contrib.learn.python.learn.datasets.base.Datasets

In [6]:
mnist.train.images.shape, mnist.train.labels.shape, mnist.test.images.shape

((55000, 784), (55000, 10), (10000, 784))

In [7]:
# 이미지 데이터가 28x28 픽셀로 처리되었고 레이블은 0 ~ 9 를 벡터화 처리가 되어있음을 알 수 있다.
mnist.train.labels[:2]

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]])

In [8]:
# 이미지 데이터는 float이다
mnist.train.images

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# 3. 데이터준비

In [9]:
# 벡터화된 레이블을 보고 원래값 0 ~ 9 를 찾아내는 방법 확인
# 힌트 : 배열 10칸 중에서 1이 존재하는 인덱스값 => 실제 숫자 값 (0 ~ 9)
import numpy as np
np.where(mnist.train.labels[:1][0])[0][0]

7

In [10]:
# 이미지 1개당 피쳐의 크기
pixels = mnist.train.images.shape[1]
pixels

784

In [11]:
# 레이블 1개당 피쳐의 크기
nums = mnist.train.labels.shape[1]
nums

10

In [12]:
# 이미지 가로크기 혹은 세로크기(정사각형이므로 동일)
pixel_wh_size = int(np.sqrt(pixels))
pixel_wh_size

28

# 4. 데이터분석(생략)

# 5. 모델 구축/모델링

## 5-1. 데이터 플로우 그래프 구축
- 상수, 변수, 플레이스홀더, 연산등의 요소를 레이어로 구축하면서 정의(Define)
- Layer 구축 및 연결(원칙: 앞단계의 출력이 뒷단계의 입력이다)
  - 입력층
  - 합성곱층 1
  - 풀링층 1
  - 합성곱층 2
  - 풀링층 2
  - 전결합층
  - 드롭아웃층
  - 출력층


## 5-2. 실행
- 데이터를 준비(betch사이즈...) -> 학습 -> 예측 -> 평가(출력)

## 5-1. 데이터 플로우 그래프 구축

### 입력층

In [0]:
# x
# 손글씨 이미지 데이터 들어오는 플레이스홀더 (외부에서 학습시 데이터가 주입된다)
# 이미지 1개를 표현 하는데 데이터가 784개의 피쳐가 필요
# 이런 이미지가 여러개 주입될 수 있다 => n개
# shape : (None , 784)
x = tf.placeholder( tf.float32, shape=(None, pixels), name= 'x') # 앞서 [None,2]랑 다른 형태 shape=(,)

### 합성곱층1

### W
- 가중치를 파라미터로 가지는 필터
- 행렬, 외부에서 주입(x), 내부적인 함수를 활용해서 한다 -> 텐서의 타입 중 Variable사용
- 두개의 합성곱층에서 w 가 각각 필요하다 => 2군데 이상 코드에서 w가 필요하다 => w를 만드는 함수를 생성하자.


In [0]:
# 가중치를 초기화 하고 W를 생성해서 리턴하는 함수 생성
# shape : 가중치를 공용파라미터로 가지는 필터(커널)의 형태
# 커널의 크기가 유동적일 수 있기 때문
# name  : 각 구성원들의 이름을 구분하기 위해서 자동생성하지 않고 직접부여
def make_Weight_Variable(shape,name):
  # 변수의 초기값 -> 절단정규분포를 통한 난수를 발생하는 함수를 활용(설정) 
  # shape이 크기에 맞춰서 배열을 만들고 함수가 계산해서 난수를 배치
  # stddev : 표준편차
  # shape 크기대로 난수가 배치되었다 (커널 크기만큼 행렬이 만들어지고 난수가 설정되었다)
  W_initValue = tf.truncated_normal( shape, stddev=0.1 )
  # 변수 생성
  W = tf.Variable( W_initValue, name='W_' + name )
  return W

In [0]:
# 편향(Bias) B도 초기화 함수 생성
# 변수로 정의하고 고정값(임계치)으로 0.1을 임시부여
def make_Bias_Variable( shape, name ):
  b_initValue = tf.constant( 0.1, shape=[shape] )
  # 변수생성
  b = tf.Variable( b_initValue, name='B_' + name )
  return b

In [16]:
# 합성곱 계층을 만드는 함수
def make_Conv2d( x,W ):
  conv2d = tf.nn.conv2d(x, W, strides=[1,1,1,1],padding='SAME') # padding=VALID
  return conv2d
"""
 tf.nn.conv2d(
    input = x -> [ batch(1회 훈련하고자 할때 이미지의 총 개수), in_height(세로크기), in_width(가로크기), in_channels(칼라에따라 다름 1 or 3) ]
    filters : 커널(=필터), 필터의 공용파라미터:필터를 구성하는 구성원의 값 W
              [filter_height, filter_width : 필터의 세로, 가로 크기 => 내 마음대로 정함
               in_channels  : 입력 채널 수 -> 1 
               out_channels : 출력 채널 수 -> 본인이 설정(1or3의 개념이 아닌 이미지를 집어 넣었을때 출력되는 이미지의 개수를 의미) ]
    strides : 정수값, 정수값의 리스트로만 구성 lenth : 1, 2나 4
              [ batch, w, h, depth ]
              통상적으로 batch, depth => 1을 사용
              w와 h 는 일반적으로 같은 값 부여
              [ 1, 1, 1, 1 ] => batch, w, h, depth
    padding : 특정맵을 보정한다. 자르다보면 외곽선 부분에서 개수가 부족하게 나올수도 있다
              이를 통상 0 으로 테두리를 치면 보정
              SAME 이라는 값을 적용
    data_format='NHWC',
    dilations=None,
    name=None
)
"""

"\n tf.nn.conv2d(\n    input = x -> [ batch(1회 훈련하고자 할때 이미지의 총 개수), in_height(세로크기), in_width(가로크기), in_channels(칼라에따라 다름 1 or 3) ]\n    filters : 커널(=필터), 필터의 공용파라미터:필터를 구성하는 구성원의 값 W\n              [filter_height, filter_width : 필터의 세로, 가로 크기 => 내 마음대로 정함\n               in_channels  : 입력 채널 수 -> 1 \n               out_channels : 출력 채널 수 -> 본인이 설정(1or3의 개념이 아닌 이미지를 집어 넣었을때 출력되는 이미지의 개수를 의미) ]\n    strides : 정수값, 정수값의 리스트로만 구성 lenth : 1, 2나 4\n              [ batch, w, h, depth ]\n              통상적으로 batch, depth => 1을 사용\n              w와 h 는 일반적으로 같은 값 부여\n              [ 1, 1, 1, 1 ] => batch, w, h, depth\n    padding : 특정맵을 보정한다. 자르다보면 외곽선 부분에서 개수가 부족하게 나올수도 있다\n              이를 통상 0 으로 테두리를 치면 보정\n              SAME 이라는 값을 적용\n    data_format='NHWC',\n    dilations=None,\n    name=None\n)\n"

In [0]:
# 합성곱층 1 생성 : 텐서보드상에 작업공간 잡기
with tf.name_scope('conv1') as scope:
  # W
  # shape = [필터세로크기, 필터가로크기, 입력채널수, 출력채널수]
  # 설정값 : 5, 5, 1, 32  => 5x5 필터를 원본이미지에서 스트라이드해서 나온 피쳐맵이 총 32개가 된다.
  W_conv1 = make_Weight_Variable( [ 5, 5, 1, 32 ], name='conv1' )

  # b
  # 가중치의 출력채널과 같은값
  b_conv1 = make_Bias_Variable( 32, 'conv1' ) 

  # x => [ batch, height, weight, channels ]
  # x => [None, 784] => [ batch, height, weight, channels ]
  # pixel_wh_size => 28 세로,가로 둘 다
  x_image = tf.reshape( x, [ -1 , pixel_wh_size, pixel_wh_size, 1] ) # batch사이즈 모를땐 -1

  # 컨볼루전 레이어 생성
  # 활성화 함수를 통과시켜서 레이어의 출력물(특성맵)을 비선형으로 보정처리한다
  # 이렇게 되면 특성맵이 활성화맵으로 되었다라고 표현할 수 있다
  h_conv1 = tf.nn.relu( make_Conv2d( x_image, W_conv1) + b_conv1 )
  # h_conv1은 합성곱층1의 출력물, 활성화맵, 다른 풀링층1의 입력데이터가 된다

In [18]:
h_conv1.shape
# 합성곱층을 통과했을때 특성맵의 크기는 원본과 같을수도 다를수도있다
# 크기를 결정하는 요인으로 -> stride, padding
# 합성곱의 핵심을 원본이미지의 특성을 추출한 이미지가 출력채널수만큼 생성된다는 점
# padding

TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(32)])

In [19]:
h_conv1

<tf.Tensor 'conv1/Relu:0' shape=(?, 28, 28, 32) dtype=float32>

### 풀링층1

- 특성맵(or 활성화맵)의 특성을 강화
- 최대풀링, 평균풀링, 최소풀링 등등 여러개 값 중 하나를 선택해서 행렬을 새로 구성(축소)
- 샘플링한다고 표현
- 크기에 관여 => 얼만큼 이동시키면서 처리할것인가? => stride, 커널에는 파라미터가 없고 (최대,최소,평균)등을 의미
- 입력 : 이전 계층 합성곱층1의 결과물 => h_conv1

In [20]:
def make_Max_Pooling( x ):
  return tf.nn.max_pool( x,ksize=[ 1,2,2,1 ],strides=[ 1,2,2,1 ],padding='SAME') 
'''
  tn.nn.maxpool(
    value   : 이전계층의 출력값, [ batch, h , w , channels ]
    ksize   : 최대값풀링을 수행하기 위한 슬라이딩윈도우(=커널)의 크기
              [ ,h,w, ]
    strides : ksize의 이동 폭, 몇 칸씩 이동 할 것인가?
              [ batch, w, h, depth ]
    padding : 보정, 크기가 맞지 않을때 사용 SAME,...
  )
  일반적으로 풀링에서 ksize와 strides는 같은값을 쓴다
'''  


'\n  tn.nn.maxpool(\n    value   : 이전계층의 출력값, [ batch, h , w , channels ]\n    ksize   : 최대값풀링을 수행하기 위한 슬라이딩윈도우(=커널)의 크기\n              [ ,h,w, ]\n    strides : ksize의 이동 폭, 몇 칸씩 이동 할 것인가?\n              [ batch, w, h, depth ]\n    padding : 보정, 크기가 맞지 않을때 사용 SAME,...\n  )\n  일반적으로 풀링에서 ksize와 strides는 같은값을 쓴다\n'

In [0]:
# 풀링층 생성
with tf.name_scope('pool1') as scope:
  h_pool1 = make_Max_Pooling(h_conv1)

In [22]:
# 이미지의 크기(사이즈)는 바뀔 수 있으나 채널수는 변함이 없다
h_pool1.shape

TensorShape([Dimension(None), Dimension(14), Dimension(14), Dimension(32)])

### 합성곱층2


In [0]:
# 레이어를 하나 더 추가한다
with tf.name_scope('conv2') as scope:
  W_conv2  = make_Weight_Variable( [ 5, 5, 32, 64 ], name='conv2' )
  b_conv2  = make_Bias_Variable( 64, 'conv2' ) 
  h_conv2 = tf.nn.relu( make_Conv2d( h_pool1, W_conv2) + b_conv2 )

### 풀링층2

In [0]:
with tf.name_scope('pool2') as scope:
  h_pool2 = make_Max_Pooling(h_conv2)

In [25]:
h_pool2

<tf.Tensor 'pool2/MaxPool:0' shape=(?, 7, 7, 64) dtype=float32>

### 전결합층

- 원본 이미지는 풀링계층을 2번 통과하면서 그 크기가 : (28 / 2 )/ 2 =7 이 되었다
- 전결합층에서는 출력층으로 가기 직전 단계로서, 행렬 데이터를 Flatten(펴주는, h , w, channel을 한줄의 데이터로 펴준다 : 동일 사이즈도 가능하고 줄여서도 가능하다)하는 것이 목적
- 최종 데이터는 [ n, size(조정) ]

In [26]:
with tf.name_scope('fully_connected') as scope:
  # 이미지 데이터 size => n 
  n = 7 * 7 * 64
  # 1024는 설정값 n(=3136) -> 1024로 줄이는 느낌으로 생각하기
  W_fc = make_Weight_Variable( [n , 1024] ,'fc')
  b_fc = make_Bias_Variable( 1024 ,'fc')
  # 이전 계층의 출력데이터를 flatten 처리 했다
  h_pool2_flatten = tf.reshape( h_pool2, [-1,n])
  h_fc = tf.nn.relu( tf.matmul( h_pool2_flatten , W_fc ) + b_fc )
h_fc  

<tf.Tensor 'fully_connected/Relu:0' shape=(?, 1024) dtype=float32>

### 드롭아웃층

- 과적합(over-fitting)을 방지하기 위해 사용
- 뉴런을 일부 죽여서 학습을 못하게 랜덤으로 처리 => 학습양을 떨어트림
- 학습 데이터에 길들여지지 않게 처리하는 목적

In [0]:
with tf.name_scope('dropout1') as scope:
  keep_prob = tf.placeholder(tf.float32)
  h_fc_drop = tf.nn.dropout( h_fc , rate=1-keep_prob )

In [28]:
h_fc_drop

<tf.Tensor 'dropout1/dropout/mul_1:0' shape=(?, 1024) dtype=float32>

### 출력층
- 이전단계의 출력 데이터를 최종레이블의 shape형태로 맞춘다
- 1024 => 10 으로 줄인다
- 이 과정에 활성화 함수를 사용할 수 있다
  - softmax 사용
    - 입력받은 값을 출력으로 0 ~ 1 사이값으로 사용
    - 모든 출력값은 0 ~ 1 사이로 정규화
    - 그 값들의 총합은 1이 되는 함수 : 특정값을 확률적으로 표현할 수 있다
    - 일반적으로 뉴럴네트워크에서는 sigmoid를 사용한다
    - 3가지 이상 범주형 분류를 하는 경우 (다항분류인경우) softmax를 가장 많이 사용한다
    - 이것이 더 좋은 결과를 낸다

In [29]:
with tf.name_scope('output') as scope:
  W_out  = make_Weight_Variable( [1024, 10], 'output' )
  b_out  = make_Bias_Variable( 10, 'output')
  y_conv = tf.nn.softmax(tf.matmul( h_fc_drop ,W_out ) + b_out)

y_conv  

<tf.Tensor 'output/Softmax:0' shape=(?, 10) dtype=float32>

In [0]:
# 이후 실행에 필요한 정답 벡터가 들어갈 플레이스홀더 요소를 준비
y_ = tf.placeholder( tf.float32 , shape=(None,nums), name='y_')

## 5-2. 실행 (학습, 예측, 평가)

### 크로스 엔트로피, 손실
- 비용(cost), 손실(loss) : 이런 지표는 원하는 결과에서 얼마나 떨어져 있는지 보여주는 지표
- 이 값(격차)을 줄이는 방향으로 전개


In [0]:
with tf.name_scope('loss') as scope:
  cross_entropy = -tf.reduce_sum( y_ * tf.log(y_conv) )

### 경사 하강법
- 크로스엔트로피의 값 (손실함수의값) 을 작아지도록 점차 반복적으로 이동시켜서 변경하는 방식
- 확률적 경사하강법
  - 무작위로 초기화한 매개변수를 손실함수값이 작아지도록 지속/반복적으로 변경처리를 수행
  

In [0]:
with tf.name_scope('sgd') as scope:
  # 1e-4 : 설정값
  optimizer = tf.train.AdamOptimizer( 1e-4 )
  train_step = optimizer.minimize( cross_entropy )

### 예측, 평가

In [0]:
with tf.name_scope('predict') as scope:
  # 최소화 하기 위해서 minimize() 사용
  # tf.argmax( y, 1) : 모델이 판단하기에 각 데이터별로 적합하다고 판단되는 레이블 값으로 취한다
  predict_step  = tf.equal( tf.argmax(y_conv,1), tf.argmax(y_,1) )
  # 정확도 => 부동소수 변환하여 평균값을 획득
  accuracy_step = tf.reduce_mean( tf.cast(predict_step, tf.float32) )

### 주입할 데이터 함수 정의
- 실행시 주입할 데이터를 편하게 구성
- 반복 학습이 수행되므로, 주입 데이터는 간편하게 구성해야 한다
- 형식 : 데이터 플로우 그래프 전체에서 placeholder를 찾아서 주입할 것들 찾기!
  - { 
      x : 
      y_ :
      keep_prob :
      } 


In [0]:
def make_FeedDict(imgs, labels, prob):
  return { x :imgs , y_ :labels , keep_prob :prob }

### 실행

In [36]:
TRAIN_COUNT = 3000 # 총 학습횟수 설정
with tf.Session() as sess:
  # 1. 텐서플로우 변수 초기화
  sess.run( tf.global_variables_initializer() )
  # 2. 주입할 데이터 준비
  test_fd = make_FeedDict(mnist.test.images, mnist.test.labels,1)
  # 3. 반복학습
  for step in range(TRAIN_COUNT):
    # 3-1. batch 사이즈(1회 훈련시 사용되는 데이터의 총양, 향후 gpu 사용시 메모리 계산) 기준으로 훈련
    batch = mnist.train.next_batch(50) # 50개씩 데이터를 사용
    # 차후에는 직접 데이터를 뽑아와서 훈련시킨다 next_batch사용x
    # 3-2. 훈련용 주입 데이터 준비
    train_fd = make_FeedDict(batch[0],batch[1], 0.5)
    # 3-3. 훈련(데이터 주입)
    _, loss = sess.run( [ train_step, cross_entropy ],feed_dict= train_fd )
    # 3-4. 특정 횟수마다 로그 출력
    if step  % 100 == 0: # 0, 100, 200, ... 2900 까지 로그
      acc = sess.run(accuracy_step, feed_dict= test_fd )
      print('%s, %s, %s' % (step,acc,loss))
  # 4. 최종결과 출력 
  f_acc = sess.run(accuracy_step, feed_dict= test_fd)
  print('최종 정답률' ,f_acc)
  # 5. 텐서보드 기록
  pass 

0, 0.0923, 448.30685
100, 0.8407, 36.231323
200, 0.9047, 37.666122
300, 0.9272, 32.428703
400, 0.9381, 11.986141
500, 0.9471, 12.993409
600, 0.9541, 16.65138
700, 0.9557, 12.985656
800, 0.9597, 13.431261
900, 0.9596, 8.477122
1000, 0.9639, 16.500628
1100, 0.9647, 13.27648
1200, 0.9687, 4.4068527
1300, 0.9675, 6.0316114
1400, 0.9705, 7.0806246
1500, 0.9718, 11.691788
1600, 0.9723, 3.423712
1700, 0.9742, 2.4454627
1800, 0.9732, 2.5718405
1900, 0.9747, 3.8618324
2000, 0.9759, 9.153113
2100, 0.9763, 6.5043373
2200, 0.9774, 5.2943907
2300, 0.9785, 6.439107
2400, 0.9771, 6.1232142
2500, 0.9784, 3.1663432
2600, 0.9793, 2.6473017
2700, 0.9802, 3.086375
2800, 0.9812, 1.7072269
2900, 0.9811, 2.9706218
최종 정답률 0.981


# 6. 시스템 통합(생략)